# [2] Text-based Statistical Research- from pdf report into statistics into csv

### This is a demo of how I use coding to do text based(pdf fromat) statistical research, this project involving techniques like OCR(Optical Character Recognition), Regular-expression and data wrangling.
### Target of this demo is to count the word frequency(in .txt) from thousands of pdf format report(100+page each) and documented into csv for future statistical research and model implementation
### <span style="color:red;"> Data is manipulated to avoid confidential information leakage!!!

In [1]:
import pandas as pd
import PyPDF2
import pdfplumber as ppl
import re

In [22]:
## get Target words
Target_word = pd.read_table('Filter.txt')
Target_word = list(Target_word['people'])
Target_word = Target_word + ['people']
##set the 7 companies
company = ['Company1','Company2', 'Company3', 'Company4', 'Company5', 'Company6', 'Company7', 'Company8', 'Company9', 'Company10']
## set years range
years = ['2017','2018','2019', '2020', '2021']

In [12]:
words_sum = dict()
for i in range(9):
    words_sum[company[i]] = {'2017':0, '2018':0, '2019':0, '2020':0, '2021':0}

In [5]:
## read 10K.pdf
import os
cwd = os.getcwd()

### Clean&Save 10K.pdf 

In [20]:
for i in company:
    for j in years:
        test = []
        with ppl.open(cwd+"\\"+i+"\\"+j+"\\10K.pdf") as pdf:
            for page in pdf.pages:
                page_text = page.extract_text().split()
                lower_text = [word.lower() for word in page_text]
                strip_word = [word.strip('()[]{}"”“:;,\'-•1234567890$.*') for word in lower_text]
                page_word = [word for word in strip_word if (word != '') and (re.search(r'\w+', word))]
                test = test + page_word
        words_sum[i][j] = test

In [21]:
### save the cleaned-up 10K report
import json
cleaned_10K_json = json.dumps(words_sum)
with open('cleaned_10K.txt', 'w') as f:
    f.write(cleaned_10K_json)

In [24]:
writer = pd.ExcelWriter('10K_count_10companies.xlsx', engine='openpyxl', mode='a')
for i in company:
    locals()[f"df_{i}"] = pd.DataFrame()
    for j in years:
        word_temp = words_sum[i][j]
        temp = pd.DataFrame()
        len_word_temp = len(word_temp)
        for t in range(764):
            temp_word = Target_word[t]
            temp.loc[t,'Word'] = temp_word
            temp.loc[t,'Count'] = word_temp.count(temp_word)
            temp.loc[t,'Density'] = word_temp.count(temp_word)/len_word_temp
        locals()[f"df_{i}"] = pd.concat([temp, locals()[f"df_{i}"]], axis = 1)
    ##save this into a sheet of the excel
    locals()[f"df_{i}"].to_excel(writer, index=False, sheet_name = i)
    writer.save()